In [1]:
import sklearn
import torch

import numpy as np
import pandas as pd

import plotly.express as px

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.cluster import adjusted_rand_score
from transformers import pipeline

## Данные

In [2]:
task = 'bts-rnc' #'active-dict' 'bts-rnc' 'wiki-wiki'

number_of_clusters = {'wiki-wiki' : 2, 'bts-rnc' : 3, 'active-dict' : 3}

In [3]:
train_df = pd.read_csv(f'../russe-wsi-kit/data/main/{task}/train.csv', delimiter='\t')

In [4]:
train_df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,90-94,"маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,69-73,Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,115-121,равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,85-90,"верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,66-70,"по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,81-86,"дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,85-90,"дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,71-77,", что актер должен иметь пять штампов-клише ""п..."
3489,3490,штамп,4,NaN,107-112,сегодняшний день в системе негосударственного ...


In [5]:
train_df = train_df.iloc[train_df['positions'].dropna().index]

In [6]:
train_df['positions'] = train_df['positions'].apply(lambda x: x.split(','))

In [7]:
train_df['positions'] = train_df['positions'].apply(lambda x: x[0].split('-'))

In [8]:
train_df[train_df['context'].apply(lambda x: len(x.split('.'))) != 1]

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,"[90, 94]","маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,"[69, 73]",Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,"[115, 121]",равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,"[85, 90]","верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,"[66, 70]","по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...
3485,3486,штамп,4,NaN,"[98, 104]",одновременно типично русской страстностью. В П...
3486,3487,штамп,4,NaN,"[81, 86]","дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,"[85, 90]","дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,"[71, 77]",", что актер должен иметь пять штампов-клише ""п..."


In [9]:
train_df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,балка,1,NaN,"[90, 94]","маленькой комнаты. Он был очень высок, наклони..."
1,2,балка,1,NaN,"[69, 73]",Пантюхин в Склифе сейчас. Он выползти на улицу...
2,3,балка,1,NaN,"[115, 121]",равнозначно обеспечивает и меланхоличную езду....
3,4,балка,1,NaN,"[85, 90]","верхняя часть закрыта, замкнута, многократно о..."
4,5,балка,1,NaN,"[66, 70]","по телевизору: наши гол забили, я вскочил от р..."
...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,"[81, 86]","дурак, но партию свою отрабатывал точно, по ус..."
3487,3488,штамп,4,NaN,"[85, 90]","дело... получается, мыслить и выражать свое мн..."
3488,3489,штамп,4,NaN,"[71, 77]",", что актер должен иметь пять штампов-клише ""п..."
3489,3490,штамп,4,NaN,"[107, 112]",сегодняшний день в системе негосударственного ...


## ruRoberta

In [10]:
def get_substitutes(context, positions, top_k=25):
        
    with_mask = ''
    
    for i, symbol in enumerate(context):
        if i == int(positions[0]):
            with_mask += '<mask>'
        elif int(positions[0]) < i < int(positions[1]):
            pass
        else:
            with_mask += symbol
            
    dicts = unmasker(with_mask, top_k=top_k)
    
    substitutes = []
    
    for d in dicts:
        substitutes.append(d['token_str'])
            
    return substitutes

In [11]:
unmasker = pipeline("fill-mask", model="sberbank-ai/ruRoberta-large")

In [ ]:
# batch_size = 16

In [13]:
%%time
# переписать с батчами
train_df['subs'] = train_df.apply(lambda x: get_substitutes(x['context'], x['positions']), axis=1)

CPU times: user 26min 40s, sys: 37.7 s, total: 27min 18s
Wall time: 27min 3s


In [14]:
train_df['subs'] = train_df['subs'].apply(lambda x: ''.join(x))

In [15]:
train_df

,context_id,word,gold_sense_id,predict_sense_id,positions,context,subs
0,1,балка,1,NaN,"[90, 94]","маленькой комнаты. Он был очень высок, наклони...",рам скоб преград глыб тяг плит толщ дек желез...
1,2,балка,1,NaN,"[69, 73]",Пантюхин в Склифе сейчас. Он выползти на улицу...,лав глыб дач Дум Госдум муз управ дум сум рас...
2,3,балка,1,NaN,"[115, 121]",равнозначно обеспечивает и меланхоличную езду....,связ тяж консол усил тяг кил креп рядами рам ...
3,4,балка,1,NaN,"[85, 90]","верхняя часть закрыта, замкнута, многократно о...",стату гир брон колод гру Библи библи но кастр...
4,5,балка,1,NaN,"[66, 70]","по телевизору: наши гол забили, я вскочил от р...",рам коллег тещ касс преград клав груш каск ст...
...,...,...,...,...,...,...,...
3486,3487,штамп,4,NaN,"[81, 86]","дурак, но партию свою отрабатывал точно, по ус...",штамп стереотип миф характер скреп прототип н...
3487,3488,штамп,4,NaN,"[85, 90]","дело... получается, мыслить и выражать свое мн...",устав протокол статус донос паспорт интернет ...
3488,3489,штамп,4,NaN,"[71, 77]",", что актер должен иметь пять штампов-клише ""п...","- - кли х штамп ст стереотип ш л/ лай "" слов и..."
3489,3490,штамп,4,NaN,"[107, 112]",сегодняшний день в системе негосударственного ...,стереотип этап пласт артефакт отход скреп аз ...


## K-means and Agglomerative clustering

In [16]:
def make_plot(df, score):
    title = df['word'].iloc[0] + f', {score}'
    labels_true = df['gold_sense_id'].apply(str).to_numpy()
    pca = sklearn.decomposition.PCA(n_components=3)
    d3 = pca.fit_transform(np.stack(df['embedding'].to_numpy()))
    d3 = pd.DataFrame(d3)
    d3['index'] = list(df.index)
    fig = px.scatter_3d(d3, x=0, y=1, z=2, color=labels_true, title=title, hover_data=['index'])
    fig.show()

In [17]:
def clustering(train_df, clusterizator_class, kwargs=None, print_every=10):
    words_info = {}
    total = 0
    ari_sum = 0
    for i, word in enumerate(set(train_df['word'])):
        df = train_df[train_df['word']==word]
        n_clusters = number_of_clusters[task]
        n_contexts = df.shape[0]
        labels_true = df['gold_sense_id'].to_numpy()
        vectorizer = TfidfVectorizer()
        corpus = list(df['subs'])
        X = vectorizer.fit_transform(corpus)
        if kwargs is None:
            clusterizator = clusterizator_class(n_clusters=n_clusters)
        else:
            clusterizator = clusterizator_class(n_clusters=n_clusters, **kwargs)
        labels_pred = clusterizator.fit_predict(X)#+1
        ari = adjusted_rand_score(labels_true, labels_pred)
        words_info[word] = {'ari' : ari, 'count' : n_contexts}
        ari_sum += ari*n_contexts
        total += n_contexts
#         if task == 'wiki-wiki' or i % print_every == 0:
#             make_plot(df, ari)            
        
    return words_info, total, ari_sum

In [18]:
words_info, total, ari_sum = clustering(train_df, KMeans, kwargs={'init' : 'random', 'max_iter' : 1000})

In [19]:
words_info

{'лев': {'ari': -0.0051654701453335665, 'count': 44},
 'поток': {'ari': -0.0038720647623624216, 'count': 136},
 'полис': {'ari': -0.007761667929806996, 'count': 142},
 'проспект': {'ari': -0.01668108995885783, 'count': 139},
 'купюра': {'ari': -0.005910116395618588, 'count': 150},
 'среда': {'ari': 0.5209547849622328, 'count': 144},
 'балка': {'ari': 0.0023199047618045155, 'count': 119},
 'жаба': {'ari': 0.34867986559534925, 'count': 121},
 'штамп': {'ari': 0.02587698211567627, 'count': 96},
 'губа': {'ari': -0.016518914805094438, 'count': 137},
 'лайка': {'ari': 0.08535507002816398, 'count': 99},
 'клетка': {'ari': 0.11430237786305952, 'count': 150},
 'опушка': {'ari': -0.006496425983627991, 'count': 148},
 'рысь': {'ari': 0.19323326028347065, 'count': 120},
 'горн': {'ari': 0.09962962962962962, 'count': 51},
 'лира': {'ari': 0.35245209044512793, 'count': 49},
 'вид': {'ari': -0.015928311074613535, 'count': 77},
 'пропасть': {'ari': -0.0005703616362916373, 'count': 127},
 'крыло': {'a

In [20]:
ari_sum/total

0.08083442346985649

In [21]:
words_info_ac, total_ac, ari_sum_ac = clustering(train_df, AgglomerativeClustering)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [22]:
words_info_ac

NameError: name 'words_info_ac' is not defined

In [23]:
ari_sum_ac/total_ac

NameError: name 'ari_sum_ac' is not defined

## Черновик

In [ ]:
train_df.iloc[139]['context'] == train_df.iloc[151]['context']

In [ ]:
train_df.iloc[139]['context']

In [ ]:
train_df.iloc[151]['context']

In [ ]:
train_df.iloc[[139, 151]]

In [ ]:
train_df.iloc[198]['context']

In [ ]:
train_df.iloc[199]['context']

In [ ]:
train_df.iloc[3]['context']

In [ ]:
train_df.iloc[106]['context']

In [ ]:
train_df.iloc[128]['context']